# Basic Context Managers: Managing Resources Efficiently

## Creator: Kuselo Ntsaluba

This tutorial was made to show some useful ways to manage resources (directories, files, code etc.) as well as reducing boilerplate, which is the repetition of the same code in different places over and over again. It is both annoying, boring and increases the wear and tear of your keyboard.

To start of with a 'testConMan.txt' file was created in the directory where this notebook file is located, consider two different ways of working with file objects:

In [1]:
#First method, if txt file does not exist it will automatically be created
F = open('testConMan.txt', 'w') 
F.write('This is a ConMan tutorial')
print(F.mode) #to show whether it is open for reading (r) or writting (w)
F.close()

w


Note that in the method above, the 'testConMan.txt' file was opened for writting purposes using the 'w' placed as a second parameter in the 'open()' command. Here we simply open the file, write to the file then close it. With this method it is important that you close the file after opening it else you will get an error.
Now consider the second method to do exactly the same thing, but instead of writting we will simply be reading the content of the file:

In [26]:
#Second method, using context manager
with open('testConMan.txt', 'r') as F:
    print(F.read())
    
print(F.closed)   

This is a ConMan tutorial
True


The advantage of the second method over the first is that you don't have to worry about the 'close' command, since after the block of code is executed the file will automatically close. It will also close the file if any exceptions or anything of that nature is encountered, this is confirmed through the use of the last print statement returning 'True'. This is the recommended way of working with a file.

This is also useful for connecting and closing databases properly as well as other uses. Here we will be using context managers to handle custom resources. 

### Further applications of Context mangers

Writting a context manager can be done in two ways:
1. Using Classes
2. Using a function with a function decorator

We will first be looking at how context managers can be used within a class scenario.

In [27]:
#creating context manager using a class
class file_open():
    def __init__(self, _filename, _mode):
        self._filename = _filename
        self._mode = _mode
        
    def __enter__(self):
        self.file = open(self._filename, self._mode)
        return self.file
    
    def __exit__(self, exc_type, exc_val, traceback):
        self.file.close()
        
with file_open('testConMan.txt', 'r') as f:
    print(f.read())
    
print(f.closed)    

This is a ConMan tutorial
True


In the above class application of context manager, we have 'init' method that runs automatically when the class object is called, initializing the variables. The 'enter' and 'exit' methods are methods that will execute on set-up and teardown of the class object. The extra parameters in the 'exit' method (exc_type, exc_val and traceback) are there just incase program throws an exception, they can be used to access that information. Looking at the output, we see the context manager works as it should.
The 'with' statement is important because it runs the code within the 'enter' method first after the 'init' method has been executed. The file is then opened within the 'enter' method and then returned to the block of code within the body of the with statement before the 'exit' method is executed on exit of the block.

Now to do something similar to the above using a function, it can be done the following way:

In [28]:
#creating context manager using contextlib
from contextlib import contextmanager

@contextmanager
def file_open(_filename, _mode):
    F = open(_filename, _mode)
    yield F
    F.close()


with file_open('testConMan.txt', 'a') as F:
    F.write(' ,This is from contextmanager')
    
with file_open('testConMan.txt', 'r') as F:
    print(F.read())    

print(F.closed)

This is a ConMan tutorial ,This is from contextmanager
True


The above function does exactly the same thing that class does (in addition to adding to the file), except it uses a decorator to do it. Here the 'open()' is the set-up of the context manager, the 'yield' statement is when the code within the 'with' statement is going to run and 'close()' is similar to the 'exit' method from the class.
However, the above isn't fully correct since we should be using the 'try' and 'finally' statements to handle errors which will be done next.

In [30]:
#try and finally blocks added
from contextlib import contextmanager

@contextmanager
def file_open(_filename, _mode):
    try:
        F = open(_filename, _mode)
        yield F
    
    finally:
        F.close()


with file_open('testConMan.txt', 'a') as F:
    F.write(' ,This is from contextmanager')
    
with file_open('testConMan.txt', 'r') as F:
    print(F.read())    

print(F.closed)

This is a ConMan tutorial ,This is from contextmanager
True


What the above addition to the code does is simply to insure that even if the code runs into an error the teardown code (the 'close()' statement) will execute in the 'finally' block.

The above function however is not very practical, since 'open()' is already a context manager within python and everything that was done within the 'with' block can be done just using the 'open()' statement. But on the upside we have shown how to replicate the functionality of the 'open()' function using a class and a function.

### Using Context Managers on directories

Now consider a situation where we are using python to do work in a number of different directories and we are constantly using command 'cd' to move between these directories. Firstly, we have to create the directories where the files will be placed, then demonstrate how this would look without a Context Manager. 

In [4]:
from contextlib import contextmanager
import os

#creating the directories //////////////////////////////////////////////////////////////////////
def createDir(_directory):
    try:
        if not os.path.exists(_directory): #checks if the directory already exists
            os.makedirs(_directory)
    except OSError:
        print ('Error: Creating directory. ' +  _directory)
        
currentwd = os.getcwd() #get current working directory
createDir('./test-dir-one/') # Creates folder in the current directory called test-dir-1
os.chdir('test-dir-one') # Changes to directory called test-dir-1
os.mknod("test1.txt") #creates test1 file in the directory
os.mknod("test2.txt")
os.mknod("test3.txt")
os.chdir(currentwd) #change back to original working directory

currentwd = os.getcwd() 
createDir('./test-dir-two/') 
os.chdir('test-dir-two') 
os.mknod("test4.txt") 
os.mknod("test5.txt")
os.mknod("test6.txt")
os.chdir(currentwd)

#cd example code (in python) without context manager //////////////////////////////////////////// 
currentwd = os.getcwd()
os.chdir('test-dir-one')
print(os.listdir()) #lists content of directory
os.chdir(currentwd)

currentwd = os.getcwd()
os.chdir('test-dir-two')
print(os.listdir())
os.chdir(currentwd)

['test2.txt', 'test1.txt', 'test3.txt']
['test5.txt', 'test4.txt', 'test6.txt']


From the above code, we see from the results the contents of both 'test-dir-one' and 'test-dir-two' directories.
The "os" library imported provides a way of using an operating system (MacOS, Ubuntu etc.) dependent on functionality. Therefore, the functions within the "os" library allow you to interface with the underlying operating system that your python version is running on. 

But as nice as the functions within the "os" library are the code is inconvenient. Reason being that we have to save our current directory, switch to another directory to do something then switch back to the orginal directory when we are done.
This is a perfect candidate for a context manager, since the code used for switching directories is repetitive. We will do it using a function.

In [6]:
from contextlib import contextmanager
import os

#cd example code rewritten
@contextmanager
def change_directory(_destination):
    try:
        currentwd = os.getcwd()
        os.chdir(_destination)
        yield
    finally:
        os.chdir(currentwd)
        
with change_directory('test-dir-one'):
    print(os.listdir())   
    
with change_directory('test-dir-two'):
    print(os.listdir())    

['test2.txt', 'test1.txt', 'test3.txt']
['test5.txt', 'test4.txt', 'test6.txt']


Note above 'yield' was used but previously we had 'yield F'. This is because in this function use of a context manage we are not working with any variables inside of our context manager (previously we worked with F using 'F.write()') and so we just use 'yield', where it will run whatever is inside the function body. You could have also printed the directory contains after creating them in the context manager above (will not be done here since it was already done previously). The advantage here is that you don't have to worry about the set-up of correct directory before doing what you want to do then the teardown to revert back to the directory you started in.

Now finally we will do the above using a class...sorry this is an exercise to see if you have understood the concepts in this tutorial, GOODLUCK ;-)

In [ ]:
##ANSWER HERE


In [7]:
##Proposed solution, ATTEMPT BEFORE YOU LOOK AT THIS
from contextlib import contextmanager
import os

#cd example code rewritten, class form
class change_directory():
    def __init__(self, _destination):
        self._destination = _destination
        
    def __enter__(self):
        self.currentwd = os.getcwd()
        os.chdir(self._destination)
    
    def __exit__(self, exc_type, exc_val, traceback):
        os.chdir(self.currentwd)
        
with change_directory('test-dir-one'):
    print(os.listdir())
    
with change_directory('test-dir-two'):
    print(os.listdir())    

['test2.txt', 'test1.txt', 'test3.txt']
['test5.txt', 'test4.txt', 'test6.txt']
